In [1]:
!apt-get update
!apt-get install -y chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [713 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,504 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.

In [2]:
# 라이브러리 불러오기
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from bs4 import BeautifulSoup
import random
import pandas as pd

In [3]:
# 데이터 양식 - 팀 이름은 스폰서 이름만 담도록
team_dict = {"한화이글스":"한화", "KIA타이거즈":"KIA", "NC다이노스":"NC", "SSG랜더스":"SSG", "LG트윈스":"LG",
             "두산베어스":"두산", "kt wiz":"KT", "롯데자이언츠":"롯데", "키움히어로즈":"키움", "삼성라이온즈":"삼성"}
# 실제 데이터를 담을 공간
pitching_info = list()

In [4]:
!apt-get update
!apt-get install -y chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

from selenium import webdriver

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')  # 사이트 안 뜨고 크롤링 하도록(--headless 뺄 경우에는 새 창이 떠서 크롤링)
chrome_options.add_argument('--no-sandbox')  # 보안 기능 중 하나인 "sandboxing"을 비활성화
chrome_options.add_argument('--disable-dev-shm-usage') #/dev/shm은 공유 메모리 파일 시스템을 나타냅니다.
#크롬이 이를 사용하면 메모리 부족 문제가 발생할 수 있는데, 이를 방지하기 위해 옵션 사용함.

chrome_path = '/usr/lib/chromium-browser/chromedriver'

driver = webdriver.Chrome(options=chrome_options)


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
chromium-chromedriver is already the newest version (1:85.0.4183.83-0ubuntu2.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 43 n

In [5]:

#드라이버 부분 수정, 위의 코드로 대체

# 액션체인 객체 생성
act = ActionChains(driver)

# 스트존 웹사이트 이동
driver.get('https://strikes.zone/game/230726SSGSSL')

# 대기
driver.implicitly_wait(3)

# 이닝별 정보 버튼 클릭
btn_inning = driver.find_element(By.XPATH, '//*[@id="__layout"]/div/div/section/section/header/div[2]/button[4]')
act.click(btn_inning)
act.perform()

In [6]:
# 게임 정보 크롤링
match_info = driver.find_element(By.XPATH, '//*[@id="__layout"]/div/div/section/section/header/div[1]')
game_info = match_info.find_elements(By.CLASS_NAME, "match__date")[0]
game_date, refree = game_info.text.replace(". ", "-").replace("투구 분포도", "").replace("(", "").replace("심판)", "").split()

# 매치 팀 정보 크롤링
match_info = driver.find_element(By.XPATH, '//*[@id="__layout"]/div/div/section/section/header/div[1]')
away_name, home_name = match_info.find_elements(By.CLASS_NAME, "match__team")[0].text.replace(" ", "").split("vs")
away_name, home_name = team_dict[away_name], team_dict[home_name]
away_name, home_name

# 이닝별 전체투구
for inning in range(2, 3):
    s_zone_path = f'//*[@id="__layout"]/div/div/section/section/section/div/div[{inning}]/div[1]/div'
    s_zone = driver.find_elements(By.XPATH, s_zone_path)[0]
    spot = s_zone.find_elements(By.TAG_NAME, "image")

    # 필요한 외부 피쳐 - game_place, piture_team, hitter_team
    # game_date, home_name, away_name, inning, x_spot, y_spot, ball_speed, ball_type, pitcher_name, pitcher_position, piture_team, hitter_name, hitter_position, judgment
    for i in spot:
        info = [game_date, home_name, away_name, inning]
        driver.implicitly_wait(random.randrange(3, 7))

        info.append(i.get_attribute("x"))
        info.append(i.get_attribute("y"))

        act.move_to_element(i)
        act.perform()
        popup = driver.find_elements(By.CLASS_NAME, "ball__detail__wrapper")

        for j in popup:
            #print(j.get_attribute("innerHTML")) # class="speed", class="result", class="stuff",
            # 구종정보
            info.append(j.find_element(By.CLASS_NAME, "speed").text)
            info.append(j.find_element(By.CLASS_NAME, "result").text)
            info.append(j.find_element(By.CLASS_NAME, "stuff").text)
            # 투수정보
            info.append(j.find_element(By.CLASS_NAME, "pitcher").find_elements(By.TAG_NAME, "span")[0].text) # pitcher_name
            info.append(j.find_element(By.CLASS_NAME, "pitcher").find_elements(By.TAG_NAME, "span")[1].text[1]) # pitcher_position
            # 타자정보
            info.append(j.find_element(By.CLASS_NAME, "hitter").find_elements(By.TAG_NAME, "span")[0].text)
            info.append(j.find_element(By.CLASS_NAME, "hitter").find_elements(By.TAG_NAME, "span")[1].text[1])
        print(info)

['2023-07-26', '삼성', 'SSG', 2, '120.97770428571428', '50.3384404395604', '139', '파울', '직구', '최채흥', 'L', '김성현', 'R']
['2023-07-26', '삼성', 'SSG', 2, '132.84036999999998', '276.5695947252747', '118', '볼', '커브', '맥카티', 'L', '김동진', 'L']
['2023-07-26', '삼성', 'SSG', 2, '46.33169999999998', '94.95695472527471', '123', '볼', '슬라이더', '맥카티', 'L', '김동진', 'L']
['2023-07-26', '삼성', 'SSG', 2, '108.55869285714287', '19.62141186813181', '138', '볼', '커터', '맥카티', 'L', '김동진', 'L']
['2023-07-26', '삼성', 'SSG', 2, '69.08205000000001', '75.6164747252747', '146', '파울', '직구', '맥카티', 'L', '김동진', 'L']
['2023-07-26', '삼성', 'SSG', 2, '75.29424142857144', '85.92728615384614', '146', '파울', '직구', '맥카티', 'L', '김동진', 'L']
['2023-07-26', '삼성', 'SSG', 2, '117.76008428571427', '180.33665758241756', '138', '플라이', '직구', '최채흥', 'L', '박성한', 'L']
['2023-07-26', '삼성', 'SSG', 2, '153.17485285714287', '266.48244043956043', '122', '볼', '커브', '맥카티', 'L', '김동진', 'L']
['2023-07-26', '삼성', 'SSG', 2, '36.71147142857144', '29.972177582417